In [19]:
# imports 

import os
import pickle
import numpy as np
import pandas as pd
from tqdm import trange

PROJECT_ROOT = os.path.abspath("..")
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
RESULTS_SA = os.path.join(PROJECT_ROOT, "experiments", "quantum_sa")
RESULTS_TN = os.path.join(PROJECT_ROOT, "experiments", "quantum_tn")

os.makedirs(RESULTS_SA, exist_ok=True)
os.makedirs(RESULTS_TN, exist_ok=True)


In [20]:
# Load Qubo inputs 

with open(os.path.join(DATA_PROCESSED, "covariance_matrices.pkl"), "rb") as f:
    scenario_covs = pickle.load(f)

market_regimes = pd.read_csv(
    os.path.join(DATA_PROCESSED, "market_regimes.csv"),
    index_col=0,
    parse_dates=True
)

dates = list(scenario_covs.keys())


In [21]:
# risk term Q matrix

def risk_term(cov):
    """
    Returns the quadratic risk term for QUBO
    """
    return cov.values


In [22]:
# Linear cost term Q matrix

def linear_change_term(prev_sel):
    """
    Linear term for |x_i - x_i_prev| in QUBO
    """
    return (1 - 2 * prev_sel)


In [23]:
# verify cardinality term Q matrix

def cardinality_penalty(n, K):
    Q = np.ones((n, n))
    linear = -2 * K * np.ones(n)
    return Q, linear


In [24]:
# Qubo builder 

def build_qubo(date, prev_sel, params):
    regime = market_regimes.loc[date, "regime"]
    cov = scenario_covs[date]["stress" if regime == "high_vol" else "base"]

    Q = params["lambda_risk"] * risk_term(cov)

    # Linear penalties
    linear = (
        params["lambda_cost"] * linear_change_term(prev_sel) +
        params["lambda_turnover"] * linear_change_term(prev_sel)
    )

    # Cardinality
    Qc, lc = cardinality_penalty(len(prev_sel), params["cardinality_max"])
    Q += params["lambda_cardinality"] * Qc
    linear += params["lambda_cardinality"] * lc

    # Put linear terms on diagonal
    for i in range(len(prev_sel)):
        Q[i, i] += linear[i]

    return Q


In [25]:
# parameters 

PARAMS = {
    "lambda_risk": 1.0,
    "lambda_cost": 0.1,
    "lambda_turnover": 0.1,
    "lambda_cardinality": 5.0,
    "cardinality_max": 10
}


Part A - simulated annealing 

In [26]:
# SA Energy function 

def qubo_energy(x, Q):
    return x @ Q @ x


In [27]:
# simulated annaealing solver 

def solve_qubo_sa(Q, num_reads=500, T0=5.0, Tf=0.01):
    n = Q.shape[0]
    best_x = None
    best_E = np.inf
    
    for _ in trange(num_reads, leave=False):
        x = np.random.randint(0, 2, size=n)
        E = qubo_energy(x, Q)
        T = T0
        
        for _ in range(200):
            i = np.random.randint(n)
            x_new = x.copy()
            x_new[i] ^= 1  # flip bit
            
            E_new = qubo_energy(x_new, Q)
            dE = E_new - E
            
            if dE < 0 or np.random.rand() < np.exp(-dE / T):
                x, E = x_new, E_new
            
            T = max(T * 0.95, Tf)
        
        if E < best_E:
            best_x, best_E = x.copy(), E
    
    return best_x, best_E


In [37]:
# run sa across all date sets 

sa_results = []

N = scenario_covs[dates[0]]["base"].shape[0]

prev_sel = np.ones(N)

for d in dates:
    Q = build_qubo(d, prev_sel, PARAMS)
    x, E = solve_qubo_sa(Q, num_reads=300)
    
    sa_results.append({
        "date": d,
        "energy": E,
        "selected": int(x.sum()),
        "selection": x.astype(int).tolist()
    })
    
    prev_sel = x

sa_df = pd.DataFrame(sa_results)
sa_df.head()


,date,energy,selected,selection
0,2021-04-30,-501.995196,10,"[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, ..."
1,2021-06-30,-501.196049,10,"[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
2,2021-08-31,-500.796983,10,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, ..."
3,2021-09-30,-500.795831,10,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
4,2021-11-30,-500.794387,10,"[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, ..."


In [31]:
# save results 

sa_df.to_csv(os.path.join(RESULTS_SA, "results.csv"), index=False)


Part B- Tensor - Network solver 

In [32]:
# Qubo -> lisng conversion 

def qubo_to_ising(Q):
    n = Q.shape[0]
    J = np.zeros((n, n))
    h = np.zeros(n)
    
    for i in range(n):
        h[i] = Q[i, i] / 2
        for j in range(i+1, n):
            J[i, j] = Q[i, j] / 4
            J[j, i] = J[i, j]
    return h, J


In [33]:
# Tensor Network style solver 

def solve_qubo_tn(Q, num_restarts=50, iters=200):
    best_x, best_E = None, np.inf

    for _ in range(num_restarts):
        h, J = qubo_to_ising(Q)
        n = len(h)
        s = np.random.choice([-1, 1], size=n)

        for _ in range(iters):
            for i in range(n):
                field = h[i] + np.dot(J[i], s)
                prob = 1 / (1 + np.exp(2 * field))
                s[i] = 1 if np.random.rand() < prob else -1

        x = (s + 1) // 2

        # Enforce cardinality
        if x.sum() > PARAMS["cardinality_max"]:
            idx = np.argsort(np.diag(Q))[:PARAMS["cardinality_max"]]
            x = np.zeros_like(x)
            x[idx] = 1

        E = x @ Q @ x
        if E < best_E:
            best_x, best_E = x.copy(), E

    return best_x, best_E



In [38]:
# Run TN across all date sets

tn_results = []

N = scenario_covs[dates[0]]["base"].shape[0]
prev_sel = np.ones(N)

for d in dates:
    Q = build_qubo(d, prev_sel, PARAMS)
    x, E = solve_qubo_tn(Q)
    
    tn_results.append({
        "date": d,
        "energy": E,
        "selected": int(x.sum()),
        "selection": x.astype(int).tolist()
    })
    
    prev_sel = x

tn_df = pd.DataFrame(tn_results)
tn_df.head()


,date,energy,selected,selection
0,2021-04-30,-501.995366,10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
1,2021-06-30,-501.995609,10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
2,2021-08-31,-501.996667,10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
3,2021-09-30,-501.996545,10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
4,2021-11-30,-501.995671,10,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."


In [39]:
# Save the results 

tn_df.to_csv(os.path.join(RESULTS_TN, "results.csv"), index=False)



In [40]:
# save as pickle 

sa_df = pd.DataFrame(sa_results)
tn_df = pd.DataFrame(tn_results)

sa_df.to_pickle(os.path.join(RESULTS_SA, "results.pkl"))
tn_df.to_pickle(os.path.join(RESULTS_TN, "results.pkl"))
